In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PySpark_Hadoop")\
.master("local[*]")\
.getOrCreate()

25/06/28 13:01:11 WARN Utils: Your hostname, dominic-To-be-filled-by-O-E-M resolves to a loopback address: 127.0.1.1; using 192.168.29.82 instead (on interface wlp2s0)
25/06/28 13:01:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/28 13:01:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql import SparkSession
from myData import test_data
from io import StringIO
data = test_data()

# Read bpm_json from string
bpm_df = spark.read.json(spark.sparkContext.parallelize(StringIO(data.bpm_json).readlines()))
bpm_df.printSchema()
bpm_df.show(truncate=False)

workouts_df = spark.read.json(spark.sparkContext.parallelize(StringIO(data.workouts_json).readlines()))
workouts_df.printSchema()
workouts_df.show(truncate=False)

users_cdc_df = spark.read.json(spark.sparkContext.parallelize(StringIO(data.users_cdc_json).readlines()))
users_cdc_df.printSchema()
users_cdc_df.show(truncate=False)

root
 |-- device_id: string (nullable = true)
 |-- heartrate: double (nullable = true)
 |-- time: string (nullable = true)

+---------------+-----------------+----------------------------+
|device_id      |heartrate        |time                        |
+---------------+-----------------+----------------------------+
|161910         |68.64985536254405|2019-12-01T11:57:31.000+0000|
|142838         |73.42208600878647|2019-12-01T11:58:03.000+0000|
|124880         |75.97373066574369|2019-12-01T11:58:03.000+0000|
|134009         |79.77338592716501|2019-12-01T11:58:20.000+0000|
|NULL           |67.91460485184962|2019-12-01T11:58:33.000+0000|
|109290         |NULL             |2019-12-01T11:58:33.000+0000|
|193806         |-69.6489747665297|2019-12-01T11:58:34.000+0000|
|Tracker #161910|65.3691488917709 |2019-12-01T11:58:42.000+0000|
+---------------+-----------------+----------------------------+

root
 |-- action: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- times

In [8]:
from pyspark.sql.functions import col

print("Cleaning data using filter")
cleanBPMData = bpm_df.filter(col("device_id").isNotNull() & col("heartrate").isNotNull() & \
                bpm_df.device_id.cast("long").isNotNull())
cleanBPMData.show(truncate=False)

Cleaning data using filter
+---------+-----------------+----------------------------+
|device_id|heartrate        |time                        |
+---------+-----------------+----------------------------+
|161910   |68.64985536254405|2019-12-01T11:57:31.000+0000|
|142838   |73.42208600878647|2019-12-01T11:58:03.000+0000|
|124880   |75.97373066574369|2019-12-01T11:58:03.000+0000|
|134009   |79.77338592716501|2019-12-01T11:58:20.000+0000|
|193806   |-69.6489747665297|2019-12-01T11:58:34.000+0000|
+---------+-----------------+----------------------------+



In [9]:
from pyspark.sql.functions import avg

print("Calculating average heartrate per device_id")
cleanBPMData.groupBy("device_id").agg(avg("heartrate").alias("avg_heartrate"))\
    .orderBy("device_id").show(truncate=False)

Calculating average heartrate per device_id
+---------+-----------------+
|device_id|avg_heartrate    |
+---------+-----------------+
|124880   |75.97373066574369|
|134009   |79.77338592716501|
|142838   |73.42208600878647|
|161910   |68.64985536254405|
|193806   |-69.6489747665297|
+---------+-----------------+



In [22]:
from pyspark.sql.functions import from_unixtime, to_date, date_format, hour, minute

from myData import test_data
data = test_data()

workouts_df = workouts_df.withColumn("readable_time", from_unixtime("timestamp")) \
                         .withColumn("date", to_date("readable_time")) \
                         .withColumn("hour", hour("readable_time")) \
                         .withColumn("minute", minute("readable_time"))

workouts_df = workouts_df.select("user_id", "timestamp", "readable_time", "date", "hour", "minute")
workouts_df.show(truncate=False)

users_df = users_cdc_df.withColumn("readable_time", from_unixtime("timestamp")) \
                   .withColumn("date", to_date("readable_time")) \
                   .withColumn("hour", hour("readable_time")) \
                   .withColumn("minute", minute("readable_time"))

users_df.select("user_id", "timestamp", "readable_time", "date", "hour", "minute").show(truncate=False)

+-------+----------+-------------------+----------+----+------+
|user_id|timestamp |readable_time      |date      |hour|minute|
+-------+----------+-------------------+----------+----+------+
|25477  |1576482430|2019-12-16 13:17:10|2019-12-16|13  |17    |
|25143  |1576501500|2019-12-16 18:35:00|2019-12-16|18  |35    |
|41732  |1576505220|2019-12-16 19:37:00|2019-12-16|19  |37    |
|49296  |1576506500|2019-12-16 19:58:20|2019-12-16|19  |58    |
|40872  |1576507900|2019-12-16 20:21:40|2019-12-16|20  |21    |
|16093  |1576508670|2019-12-16 20:34:30|2019-12-16|20  |34    |
|14633  |1576508160|2019-12-16 20:26:00|2019-12-16|20  |26    |
+-------+----------+-------------------+----------+----+------+

+-------+----------+-------------------+----------+----+------+
|user_id|timestamp |readable_time      |date      |hour|minute|
+-------+----------+-------------------+----------+----+------+
|31362  |1555539580|2019-04-18 03:49:40|2019-04-18|3   |49    |
|31362  |1555539580|2019-04-18 03:49:40

In [4]:
workouts_df.groupBy("action").count().orderBy("action").show(truncate=False)

+------+-----+
|action|count|
+------+-----+
|NULL  |1    |
|start |3    |
|stop  |3    |
+------+-----+



In [12]:
from pyspark.sql.functions import col

workout_df = workouts_df.filter(col("workout_id").isNotNull() & workouts_df.session_id.cast("long").isNotNull())
workout_df.show(truncate=False)


+------+----------+----------+-------+----------+-------------------+----------+----+------+
|action|session_id|timestamp |user_id|workout_id|readable_time      |date      |hour|minute|
+------+----------+----------+-------+----------+-------------------+----------+----+------+
|stop  |9         |1576482430|25477  |9         |2019-12-16 13:17:10|2019-12-16|13  |17    |
|start |7         |1576501500|25143  |21        |2019-12-16 18:35:00|2019-12-16|18  |35    |
|start |7         |1576505220|41732  |24        |2019-12-16 19:37:00|2019-12-16|19  |37    |
|stop  |69        |1576506500|49296  |33        |2019-12-16 19:58:20|2019-12-16|19  |58    |
|NULL  |508       |1576508670|16093  |27        |2019-12-16 20:34:30|2019-12-16|20  |34    |
+------+----------+----------+-------+----------+-------------------+----------+----+------+



In [13]:
users_df.show(truncate=False)

+---------------------------------------------------------+----------+-----------+------+---------+---+----------+-----------+-------+-------------------+----------+----+------+
|address                                                  |dob       |first_name |gender|last_name|sex|timestamp |update_type|user_id|readable_time      |date      |hour|minute|
+---------------------------------------------------------+----------+-----------+------+---------+---+----------+-----------+-------+-------------------+----------+----+------+
|{Los Angeles, CA, 4791 Nathan Turnpike Suite 540, 90024} |02/01/2001|Shelley    |F     |Andrews  |F  |1555539580|new        |31362  |2019-04-18 03:49:40|2019-04-18|3   |49    |
|{Los Angeles, CA, 4791 Nathan Turnpike Suite 540, 90024} |02/02/2001|Andrew     |F     |Shelley  |F  |1555539580|update     |31362  |2019-04-18 03:49:40|2019-04-18|3   |49    |
|{Canyon Country, CA, 973 Norman Mountain Apt. 290, 91386}|05/10/1965|Jennifer   |F     |Perez    |F  |1559688

In [ ]:
from pyspark.sql.functions import count, col

usersRecord = users_df.groupBy(users_cdc_df.columns).agg(count("*").alias("count")).filter(col("count") == 1) \
.filter(col("user_id").isNotNull() & col("update_type").isNotNull())
usersRecord.show(truncate=False)

usersRecord = users_df.groupBy(users_df.columns).agg(count("*").alias("count")) \
    .filter((col("count") == 1) & col("user_id").isNotNull() & col("update_type").isNotNull())


+---------------------------------------------------------+----------+----------+------+---------+---+----------+-----------+-------+-------------------+----------+----+------+-----+
|address                                                  |dob       |first_name|gender|last_name|sex|timestamp |update_type|user_id|readable_time      |date      |hour|minute|count|
+---------------------------------------------------------+----------+----------+------+---------+---+----------+-----------+-------+-------------------+----------+----+------+-----+
|{Los Angeles, CA, 4791 Nathan Turnpike Suite 540, 90024} |02/01/2001|Shelley   |F     |Andrews  |F  |1555539580|new        |31362  |2019-04-18 03:49:40|2019-04-18|3   |49    |1    |
|{Los Angeles, CA, 4791 Nathan Turnpike Suite 540, 90024} |02/02/2001|Andrew    |F     |Shelley  |F  |1555539580|update     |31362  |2019-04-18 03:49:40|2019-04-18|3   |49    |1    |
|{Canyon Country, CA, 973 Norman Mountain Apt. 290, 91386}|05/10/1965|Jennifer  |F   

In [42]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc
usersRecordFilter = usersRecord.withColumn("row_number",
    row_number().over(Window.partitionBy("user_id").orderBy(desc("date"))))

usersRecord.withColumn("row_number",
    row_number().over(Window.partitionBy("user_id").orderBy(desc("date")))).collect()

usersRecordFilter.show()

+--------------------+----------+----------+------+---------+---+----------+-----------+-------+-------------------+----------+----+------+-----+----------+
|             address|       dob|first_name|gender|last_name|sex| timestamp|update_type|user_id|      readable_time|      date|hour|minute|count|row_number|
+--------------------+----------+----------+------+---------+---+----------+-----------+-------+-------------------+----------+----+------+-----+----------+
|{Altadena, CA, 27...|03/07/1941|     Casey|     M|    Miles|  M|1566185600|        new| #36644|2019-08-19 09:03:20|2019-08-19|   9|     3|    1|         1|
|{Los Angeles, CA,...|02/01/2001|   Shelley|     F|  Andrews|  F|1555539580|        new|  31362|2019-04-18 03:49:40|2019-04-18|   3|    49|    1|         1|
|{Los Angeles, CA,...|02/02/2001|    Andrew|     F|  Shelley|  F|1555539580|     update|  31362|2019-04-18 03:49:40|2019-04-18|   3|    49|    1|         2|
|{Canyon Country, ...|05/10/1965|  Jennifer|     F|    Per